---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [25]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [26]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [27]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    towns = pd.read_csv('university_towns.txt',sep='\n')
    towns['State']=' '
    curs = 'Alabama'
    for i,row in towns.iterrows():
        if '[edit]' in row['Alabama[edit]']:
            curs=row['Alabama[edit]']
        towns.at[i,'State']=curs
    towns.columns=['RegionName','State']
    towns = towns[towns['RegionName'].str.contains('[edit]',regex=False)==0]
    towns['State'] = towns['State'].str.replace(r'\[.*', '')
    towns['RegionName'] = towns['RegionName'].str.replace(r'\s\(.*', '')
    statelist=towns['State']
    regionlist=towns['RegionName']
    ans = pd.DataFrame([statelist,regionlist])
    ans=ans.T
    return ans
#get_list_of_university_towns()

In [28]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df=pd.read_excel('gdplev.xls',skiprows=5)
    #df=df[df['Unnamed: 0']>=2000]
    df=df.rename(columns={'Unnamed: 4':'Quaters','GDP in billions of chained 2009 dollars.1':'GDP'})#'GDP in billions of chained 2009 dollars.2':'Q2',})
    df2=df[['Quaters','GDP']]
    df3=pd.DataFrame(df2)
    #f3=df3.set_index('Quaters')
    df3=df3[214:]
    #return df3
    ans=[]
    rec=0
    for i in range(len(df3)-2):
        if rec==0 and (df3.iloc[i+1,1]<df3.iloc[i,1] and df3.iloc[i+2,1]<df3.iloc[i+1,1]):
            ans.append(df3.iloc[i+1,0])
            rec=1
        if rec==1 and (df3.iloc[i,1]<df3.iloc[i+1,1] and df3.iloc[i+1,1]<df3.iloc[i+2,1]):
            rec=0
            #print(df3.iloc[i,0])
    return ans[0]
#get_recession_start()

In [29]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    df=pd.read_excel('gdplev.xls',skiprows=5)
    #df=df[df['Unnamed: 0']>=2000]
    df=df.rename(columns={'Unnamed: 4':'Quaters','GDP in billions of chained 2009 dollars.1':'GDP'})#'GDP in billions of chained 2009 dollars.2':'Q2',})
    df2=df[['Quaters','GDP']]
    df3=pd.DataFrame(df2)
    #f3=df3.set_index('Quaters')
    df3=df3[214:]
    #return df3
    ans=[]
    rec=0
    for i in range(len(df3)-2):
        if rec==0 and (df3.iloc[i+1,1]<df3.iloc[i,1] and df3.iloc[i+2,1]<df3.iloc[i+1,1]):
            rec=1
        if rec==1 and (df3.iloc[i,1]<df3.iloc[i+1,1] and df3.iloc[i+1,1]<df3.iloc[i+2,1]):
            rec=0
            ans.append(df3.iloc[i+2,0])
            #print(df3.iloc[i,0])
    return ans[0]
#get_recession_end()

In [30]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    df=pd.read_excel('gdplev.xls',skiprows=5)
    #df=df[df['Unnamed: 0']>=2000]
    df=df.rename(columns={'Unnamed: 4':'Quaters','GDP in billions of chained 2009 dollars.1':'GDP'})#'GDP in billions of chained 2009 dollars.2':'Q2',})
    df2=df[['Quaters','GDP']]
    df3=pd.DataFrame(df2)
    #f3=df3.set_index('Quaters')
    df3=df3[247:252]
    ans= df3['GDP'].idxmin()
    return df3.iloc[ans-247,0]
#get_recession_bottom()

In [31]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df=pd.read_csv("City_Zhvi_AllHomes.csv")
    df2=df.loc[:,'2000-01':'2016-08']
    df2['State']=df['State']
    df2['RegionName']=df['RegionName']
    for i,_ in df2.iterrows():
        df2.at[i,'State']=states[df2.at[i,'State']] 
    df2=df2.set_index(['State','RegionName'])
    col=[]
    for year in range(2000,2016):
        y=str(year)
        df2[y+'q1']=(df2[y+'-01']+df2[y+'-02']+df2[y+'-03'])/3
        df2[y+'q2']=(df2[y+'-04']+df2[y+'-05']+df2[y+'-06'])/3
        df2[y+'q3']=(df2[y+'-07']+df2[y+'-08']+df2[y+'-09'])/3
        df2[y+'q4']=(df2[y+'-10']+df2[y+'-11']+df2[y+'-12'])/3
        col.append(y+'q1')
        col.append(y+'q2')
        col.append(y+'q3')
        col.append(y+'q4')
    df2['2016q1']=(df2['2016-01']+df2['2016-02']+df2['2016-03'])/3
    df2['2016q2']=(df2['2016-04']+df2['2016-05']+df2['2016-06'])/3
    df2['2016q3']=(df2['2016-07']+df2['2016-08'])/2
    col.append('2016q1')
    col.append('2016q2')
    col.append('2016q3')
    df2=df2[col]
    return df2
    return "ANSWER"
#convert_housing_data_to_quarters()

In [34]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    recbeg=get_recession_start()
    recend=get_recession_end()
    recbt=get_recession_bottom()
    df=convert_housing_data_to_quarters()
    df=df.loc[:,'2008q2':recend]
    df=df.dropna()
    df2=get_list_of_university_towns()
    #unitown=pd.merge(df,df2,how='inner',left_on=['State','RegionName'],right_on=['State','RegionName'])
    #unitown = unitown.set_index(['State','RegionName'])
    unitown_list=df2.to_records(index=False).tolist()
    unitown=df.loc[unitown_list]
    nonunitown=df.loc[~df.index.isin(unitown_list)]
    #frames=[df,unitown]
    #nonunitown = pd.concat(frames)
    #nonunitown = nonunitown.drop_duplicates(keep=False)
    #return (unitown,nonunitown)
    unitownstat=(unitown['2008q2']).div(unitown[recbt])
    nonunitownstat=(nonunitown['2008q2']).div(nonunitown[recbt])
    unitownstat=unitownstat.dropna()
    nonunitownstat=nonunitownstat.dropna()
    unitownstat=unitownstat.reset_index()[0]
    nonunitownstat=nonunitownstat.reset_index()[0]
    test=ttest_ind(unitownstat,nonunitownstat)[1]
    ans=(True,test,'university town')
    return ans
#run_ttest()


c:\users\alphastar\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike


(True, 0.0027134586648562414, 'university town')